# 2 Собираем датасет транзакций

## Выделяем фичи справочников

Как мы увидели ранее, у на есть два справочника. Каждый из справочников состоит их 2х полей: ключ и описание.

Посмотрим как выглядят значения описания значений справочника и попробуем выделить из описания дополнительные фичи.

### Справочник MCC

In [6]:
import pandas as pd
pd.set_option('max_colwidth', 400)

In [2]:
tr_mcc_codes = pd.read_csv('kaggle_datasets/tr_mcc_codes.csv', delimiter=';')
tr_mcc_codes.sample(7, random_state=7)

,mcc_code,mcc_description
182,9399,"Правительственные услуги, нигде ранее не классифицируемые"
179,9211,"Судовые выплаты, включая алименты и детскую поддержку"
131,7011,"Отели, мотели, базы отдыха, сервисы бронирования"
24,4900,"Коммунальные услуги — электричество, газ, санитария, вода"
65,5541,Станции техобслуживания
104,5947,"Магазины открыток, подарков, новинок и сувениров"
32,5074,Оборудование для водопровода и отопительной системы


**Что мы видим**

1. **Ключом справочника является [MCC код](https://ru.wikipedia.org/wiki/Merchant_Category_Code)**. Это международный справочник, в котором перечислены значения отраслей и категорий точек продаж. Последние две цифры кода - категория, а первые 1-2 символа - отрасль.

2.  **Описание категории сложносоставное.** Необходимо привести его упростить.

Выделим из названия основу (до знаков запятой, тире и слэша) и отдельно выделим макроотрасль MCC (первые две цифиы ).

In [3]:
# сохраним текущее значение описания в отдельном поле
tr_mcc_codes['mcc_full_description'] = tr_mcc_codes['mcc_description']


# сформируем описание МСС для дальнейшей работы с ним

## подготовим функцию, выделяющую первую часть значения (до разделителя)
def extract_text_before_delimiter(value):
    delimiters = [',', '—', '/']
    for delimiter in delimiters:
        if delimiter in value:
            value = value.split(delimiter)[0].strip()
    return value

## пройдемся по всем строкам функцией, выделяющей текст до разделителя
for index, row in tr_mcc_codes.iterrows():
    tr_mcc_codes.at[index, 'mcc_description'] = extract_text_before_delimiter(row['mcc_description'])

## добавим к сокращенному описанию значение кода MCC, чтобы исключить дубликаты в этом поле
tr_mcc_codes['mcc_description'] = tr_mcc_codes['mcc_description'] + ' (' + tr_mcc_codes['mcc_code'].astype('str') + ')' 


# выделяем группу MCC отдельной фичей
tr_mcc_codes['mcc_group'] = tr_mcc_codes['mcc_code'].astype('str').apply(lambda x: x[:2])


# сохраним получившийся справочник отдельным файлом
import os
os.makedirs('./datasets', exist_ok=True) # создадим папку для сохранения

tr_mcc_codes.to_csv('datasets/tr_mcc_codes_featured.csv', index=False)
tr_mcc_codes.head(5)

,mcc_code,mcc_description,mcc_full_description,mcc_group
0,742,Ветеринарные услуги (742),Ветеринарные услуги,74
1,1711,Генеральные подрядчики по вентиляции (1711),"Генеральные подрядчики по вентиляции, теплоснабжению, и водопроводу",17
2,1731,Подрядчики по электричеству (1731),Подрядчики по электричеству,17
3,1799,Подрядчики (1799),"Подрядчики, специализированная торговля — нигде более не классифицированные",17
4,2741,Разнообразные издательства (2741),Разнообразные издательства/печатное дело,27


Справочник MCC готов

### Справочник типов категорий

In [8]:
tr_types = pd.read_csv('kaggle_datasets/tr_types.csv', delimiter=';')
tr_types.sample(7, random_state=7)

,tr_type,tr_description
90,3810,н/д
76,8140,Операции по утраченной карте
37,6210,Возврат покупки.POS Зарубеж. ТУ
11,4040,Плата за взнос наличных через POS (в своем тер.банке)
70,8001,Установление расх. лимита по
146,2432,Перевод с карты на счет физ.лица в другом банке через Сбербанк ОнЛ@йн
118,2330,Списание с карты по операции “перевода с карты на карту” через АТМ (в пределах одного тер.банка)



Номер типа операции является внутренним справочником Сбера. Не будем его использовать. 

В описании типа транзакции находится множество информации. Достанем из описания ключевые параметра типа транзакции:
- **устройство**, в котором она была произведена (банкомат, интернет-банк, смс-банк или филиал)
- является ли транзакция операцией **с наличными**, комиссией за использование наличных или безналичная операция
- **регион** операции (зарубежная или российская операция) и банк (домашний банк или "роуминг")
- собственно **тип операции** исходя из названия

In [10]:
device_type_map = {
    'АТМ'                                       : 'АТМ',
    'Банкомат'                                  : 'ATM',
    'через УС'                                  : 'ATM',
    'POS'                                       : 'POS',
    'ТУ'                                        : 'POS',
    'через Мобильный банк'                      : 'SMS',
    'посредством мобильной связи'               : 'mobile',
    'ВСП'                                       : 'VSP',
    'в подразд'                                 : 'VSP',
    'Сбербанк ОнЛ@йн'                           : 'SBOL',
    'через Интернет'                            : 'SBOL',
    'автомат'                                   : 'auto',
} 


cash_type_map = {
    'Плата за получение наличных'               : 'cash_trx_payment',
    'Плата за взнос наличных'                   : 'cash_trx_payment',
    'экстренной выдачи наличных'                : 'cash_trx_payment',
    'Взнос наличных'                            : 'cash_trx',
    'Выдача наличных'                           : 'cash_trx',
}


region_map = {
    'Зарубеж'                                   : 'foreign_trx',
    'Россия'                                    : 'other_bank_trx',
    'в чужом АТМ'                               : 'other_bank_trx',
    'из одного ТБ в другой ТБ'                  : 'sberbank_trx',
    'СБ РФ'                                     : 'sberbank_trx',
    'АТМ Сбербанка'                             : 'sberbank_trx',
    'Сбербанк'                                  : 'home_terbank_trx',
    'в своем тер.банке'                         : 'home_terbank_trx',
    'в пределах одного тер.банка'               : 'home_terbank_trx',
    'в своем ТБ'                                : 'home_terbank_trx',
    'в пределах одного филиала'                 : 'home_terbank_trx',
    'в других ТБ'                               : 'other_terbank_trx',
    'в другом тер.банк'                         : 'other_terbank_trx',
}


trx_type_map = {
    'Возврат'                                   : 'Возврат',
    'Перевод'                                   : 'B2C перевод',
    'перевод'                                   : 'B2C перевод',
    'Плата за'                                  : 'Комиссия',
    'Взнос наличных'                            : 'Взнос наличных',
    'Оплата услуг банка'                        : 'Комиссия',
    'Покупка'                                   : 'POS покупка',
    'Пополнение счета'                          : 'Пополнение',
    'Платеж с карты '                           : 'Платеж',
    'Оплата услуги'                             : 'Платеж',
    'Выдача наличных'                           : 'Выдача наличных',
    'Погашение кредита'                         : 'Погашение кредита',
    'Наличные'                                  : 'Выдача наличных',
    'Списание'                                  : 'Списание',
    'списание'                                  : 'Списание',
    'Списание с карты по операции “перевода'    : 'B2C перевод',
    'Комиссия'                                  : 'Комиссия',
    'пополнения зарплатных '                    : 'Заработная плата',
    'перечисления процентов'                    : 'Начисление процентов',
}


def dict_enrichment(dict, field, default_val):
    """
    Проверяет в каждом поле справочника наличие значения `text` и
     подставляет в поле `field` значение из маппера `dict`
    """
    for text in dict:
        tr_types.loc[tr_types['tr_description'].str.contains(text), field] = dict[text]
    tr_types[field] = tr_types[field].fillna(default_val)


dict_enrichment(device_type_map,    'device',           'unknown')
dict_enrichment(cash_type_map,      'cash_relation',    'unrelated')
dict_enrichment(region_map,         'region',           'unrelated')
dict_enrichment(trx_type_map,       'type',             'other')

tr_types.to_csv('datasets/tr_types_featured.csv', index=False)
tr_types.sample(5, random_state=11)


C:\Users\Darya Alekseeva\AppData\Local\Temp\ipykernel_16728\738363661.py:72: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'АТМ' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tr_types.loc[tr_types['tr_description'].str.contains(text), field] = dict[text]
C:\Users\Darya Alekseeva\AppData\Local\Temp\ipykernel_16728\738363661.py:72: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'cash_trx_payment' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tr_types.loc[tr_types['tr_description'].str.contains(text), field] = dict[text]
C:\Users\Darya Alekseeva\AppData\Local\Temp\ipykernel_16728\738363661.py:72: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'foreign_trx' has dtype incompatible with float64, 

,tr_type,tr_description,device,cash_relation,region,type
119,2331,Списание с карты по операции “перевода с карты на карту” через АТМ (из одного ТБ в другой ТБ),АТМ,unrelated,sberbank_trx,B2C перевод
137,2412,Перевод с карты на счет др.лица в одном тер. банке через Сбербанк ОнЛ@йн,SBOL,unrelated,home_terbank_trx,B2C перевод
126,2352,Погашение кредита с карты через АТМ,АТМ,unrelated,unrelated,Погашение кредита
65,7090,Межфилиальные пополнения зарплатных карт,unknown,unrelated,unrelated,Заработная плата
88,2440,н/д,unknown,unrelated,unrelated,other
